In [1]:
import sys



In [2]:
!pip install duckdb==0.7.1 \
duckdb-engine \
watermark \
jupysql \
sqlalchemy \
python-snappy \
pyarrow \
memray \
pandas  \
ipywidgets  \
matplotlib \
gensim \
nltk \
plotly \
redis==4.5.3 \
jupyter-black \
sentence_transformers \
redis \
jupyter_black

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 55.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 76.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 87.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 53.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 82.0 

In [3]:
# Autoformat cells on run
import jupyter_black
import pandas as pd

jupyter_black.load()

In [4]:
# set log level for model training
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(
    format="%(levelname)s - %(asctime)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

In [10]:
embeddings = pd.read_parquet("~/ssl/2023071112_training.parquet")

In [11]:
embeddings

,review_text,title,link,description,average_rating,author,text_reviews_count,sentence
0,Read more of this review @ The Book Return Blo...,The Boston Girl,https://www.goodreads.com/book/show/22450859-t...,From the New York Times bestselling author of ...,3.79,Anita Diamant,6850,Read more of this review @ The Book Return Blo...
1,"The storyline sounded intriguing, but I just c...",Children of the Jedi,https://www.goodreads.com/book/show/555313.Chi...,Callista is a brave Jedi warrior who gave her ...,3.26,Barbara Hambly,119,"The storyline sounded intriguing, but I just c..."
2,Some good short stories. I particularly liked ...,American Girls About Town,https://www.goodreads.com/book/show/893879.Ame...,This title is an anthology by American women w...,3.47,Jennifer Weiner,5,Some good short stories. I particularly liked ...
3,Not thorough enough.,The Pagan Book of Days: A Guide to the Festiva...,https://www.goodreads.com/book/show/1020515.Th...,,3.83,Nigel Pennick,6,Not thorough enough.The Pagan Book of Days: A ...
4,I loved the way this book used magic and myste...,"Circles in the Stream (Avalon: Web of Magic, #1)",https://www.goodreads.com/book/show/2354843.Ci...,"Three very different young teens, Emily, the s...",4.08,Rachel Roberts,132,I loved the way this book used magic and myste...
...,...,...,...,...,...,...,...,...
840849,*Read for school. \n I hated this book SO MUCH...,The Mask of Gold,https://www.goodreads.com/book/show/12796641-t...,,1.98,Alan A. McLean,202,*Read for school. \n I hated this book SO MUCH...
840850,tyyyyyyyyyyyyyyyyyyyyyt,The Mask of Gold,https://www.goodreads.com/book/show/12796641-t...,,1.98,Alan A. McLean,202,tyyyyyyyyyyyyyyyyyyyyytThe Mask of GoldAlan A....
840851,'qwl yh bs !!! yl y zbl@ bdhkrytk lzbl@ .... q...,The Mask of Gold,https://www.goodreads.com/book/show/12796641-t...,,1.98,Alan A. McLean,202,'qwl yh bs !!! yl y zbl@ bdhkrytk lzbl@ .... q...
840852,There was the promise of some fascinating idea...,Tapping Into Past Lives: Heal Soul Traumas and...,https://www.goodreads.com/book/show/23869887-t...,,3.69,Jenny Johnston,3,There was the promise of some fascinating idea...


In [12]:
corpus = embeddings["sentence"].tolist()
titles = embeddings["title"].tolist()
author = embeddings["author"].tolist()
link = embeddings["link"].tolist()
review_count = embeddings["text_reviews_count"].tolist()
indices = embeddings.index.tolist()

In [13]:
from sentence_transformers import SentenceTransformer, util

#  A common value for BERT & Co. are 512 word pieces, which correspond to about 300-400 words (for English).
# Longer texts than this are truncated to the first x word pieces.
# By default, the provided methods use a limit fo 128 word pieces, longer inputs will be truncated
# the runtime and the memory requirement grows quadratic with the input length - we'll have to play around with this

# Change the length to 200
model = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-v3")
model.max_seq_length = 200


corpus_embeddings = model.encode(
    corpus, show_progress_bar=True, device="cuda", convert_to_numpy=False
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/msmarco-distilbert-base-v3


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/26277 [00:00<?, ?it/s]

In [ ]:
embeddings_list = [x.tolist() for x in corpus_embeddings]
embedding_tuple = list(
    zip(titles, indices, author, link, review_count, embeddings_list)
)

In [ ]:
from tqdm import tqdm
import numpy as np
import pyarrow as pa
import pandas as pd

df = pd.DataFrame(
    embedding_tuple,
    columns=["title", "index", "author", "link", "review_count", "embeddings"],
)

In [16]:
df

,title,index,author,link,review_count,embeddings
0,The Boston Girl,0,Anita Diamant,https://www.goodreads.com/book/show/22450859-t...,6850,"[0.3218096196651459, -0.10958044230937958, -0...."
1,Children of the Jedi,1,Barbara Hambly,https://www.goodreads.com/book/show/555313.Chi...,119,"[-0.35003960132598877, -0.19994723796844482, 0..."
2,American Girls About Town,2,Jennifer Weiner,https://www.goodreads.com/book/show/893879.Ame...,5,"[-0.04687519371509552, -0.531970202922821, 0.3..."
3,The Pagan Book of Days: A Guide to the Festiva...,3,Nigel Pennick,https://www.goodreads.com/book/show/1020515.Th...,6,"[0.007908889092504978, -0.2877320945262909, 0...."
4,"Circles in the Stream (Avalon: Web of Magic, #1)",4,Rachel Roberts,https://www.goodreads.com/book/show/2354843.Ci...,132,"[0.4315156042575836, -0.26592448353767395, 0.3..."
...,...,...,...,...,...,...
840849,The Mask of Gold,840849,Alan A. McLean,https://www.goodreads.com/book/show/12796641-t...,202,"[0.056099820882081985, 0.47604134678840637, -0..."
840850,The Mask of Gold,840850,Alan A. McLean,https://www.goodreads.com/book/show/12796641-t...,202,"[0.7861809730529785, 1.0913476943969727, 0.083..."
840851,The Mask of Gold,840851,Alan A. McLean,https://www.goodreads.com/book/show/12796641-t...,202,"[0.9624336957931519, 0.7762323617935181, 0.390..."
840852,Tapping Into Past Lives: Heal Soul Traumas and...,840852,Jenny Johnston,https://www.goodreads.com/book/show/23869887-t...,3,"[0.5977463126182556, -0.2535926103591919, 0.06..."


In [17]:
fields = [
    ("title", pa.string()),
    ("index", pa.int64(), False),
    ("author", pa.string()),
    ("link", pa.string()),
    ("review_count", pa.string()),
    ("embeddings", pa.large_list(pa.float64())),
]
schema = pa.schema(fields)

df.to_parquet(
    "20230711_learned_embeddings.snappy",
    engine="pyarrow",
    compression="snappy",
    schema=schema,
)